# Vosk CLI stderr output to CTM

> "Because it was quicker than looking at the API examples"

- branch: master
- comments: false
- categories: [vosk, ctm, kludge]

In [23]:
VOSKDIR = "/Users/joregan/hsi-vosk/"
GOOGDIR = "/Users/joregan/Playing/hsi_google/"

In [24]:
from pathlib import Path

vosk_path = Path(VOSKDIR)
goog_path = Path(GOOGDIR)

In [5]:
def get_recognition(filename):
    segments = []
    with open(filename) as inf:
        for line in inf.readlines():
            if line.startswith("INFO:root:{'result':"):
                text = line.strip()[10:]
                data = eval(text)
                segments.append(data)
    return segments

In [28]:
recs = get_recognition(EG)

In [31]:
print(recs[0]['result'])

[{'conf': 1.0, 'end': 1.11, 'start': 0.9, 'word': 'for'}, {'conf': 1.0, 'end': 1.41, 'start': 1.17, 'word': 'sure'}]


In [1]:
def clean_filename(filename):
    if type(filename) == str:
        filename = Path(filename).stem
    elif type(filename) == Path:
        filename = filename.stem
    return filename

In [9]:
rec = get_recognition("/Users/joregan/hsi-vosk/hsi_1_0515_209_001_inter.wav.vosk")

In [12]:
from dataclasses import dataclass, field

@dataclass(frozen=True)
class TimedWord:
    text: str = field(compare=True)
    start: float = field(compare=True)
    end: float = field(compare=True)

In [ ]:
rec

In [17]:
def get_arrayrefs_from_vosk(rec):
    pointers = {}
    for c, data in enumerate(rec):
        for w in data["result"]:
            tw = TimedWord(w["word"], w["start"], w["end"])
            pointers[tw] = c
    return pointers

In [20]:
def get_arrayrefs_from_google(data):
    c = 0
    pointers = {}
    for result in data["results"]:
        if not "alternatives" in result:
            continue
        if not len(result["alternatives"]) == 1:
            continue
        if not "words" in result["alternatives"][0]:
            continue
        for w in result["alternatives"][0]["words"]:
            tw = TimedWord(w["word"], w["startTime"], w["endTime"])
            pointers[tw] = c
        c += 1
    return pointers

{TimedWord(text='for', start=0.9, end=1.11): 0,
 TimedWord(text='sure', start=1.17, end=1.41): 0,
 TimedWord(text='hello', start=2.7, end=2.94019): 1,
 TimedWord(text='on', start=2.94019, end=3.12): 1,
 TimedWord(text='ya', start=3.12, end=3.36): 1,
 TimedWord(text='hello', start=3.81, end=4.59): 1,
 TimedWord(text="you're", start=5.31, end=5.52): 2,
 TimedWord(text="you're", start=5.55, end=5.73): 2,
 TimedWord(text='like', start=5.73, end=5.94): 2,
 TimedWord(text='floating', start=5.94, end=6.51): 2,
 TimedWord(text='in', start=6.51, end=6.63): 2,
 TimedWord(text='the', start=6.63, end=6.69): 2,
 TimedWord(text='room', start=6.69, end=7.14): 2,
 TimedWord(text='and', start=8.46, end=8.700373): 2,
 TimedWord(text='floating', start=8.700373, end=9.66): 2,
 TimedWord(text='your', start=9.72, end=9.80909): 2,
 TimedWord(text='shoes', start=9.80909, end=10.220186): 2,
 TimedWord(text='outside', start=10.23, end=10.77): 2,
 TimedWord(text='steam', start=11.4, end=12.06): 2,
 TimedWord(tex

In [54]:
# for file in Path(DIR).glob("*.vosk"):
#     recs = get_recognition(str(file))

#     ctmlines = []
#     for rec in recs:
#         res = rec["result"]
#         ctmlines += [kaldi_word_to_ctm_line(x, EG) for x in res]
#     outfile = clean_filename(str(file)) + ".ctm"
